In [68]:
FILE = 'input.txt'
with open(FILE) as f:
    lines = f.readlines()

In [65]:
import math
from itertools import combinations
from functools import reduce

points = [tuple(map(int, line.strip().split(','))) for line in lines]
edges = sorted(combinations(points, 2), key=lambda pair: math.dist(*pair))[:10 if 'small' in FILE else 1000]
# Traverse graph and return order
def traverse_graph(edges) -> int:
    visited = set()
    to_visit = [edges[0][0]]
    while len(to_visit) > 0:
        current = to_visit.pop()
        edges_to_remove = []
        for edge in filter(lambda e: e[0] == current and e[1] not in visited or e[1] == current and e[0] not in visited, edges):
            to_visit.append(edge[0] if edge[1] == current else edge[1])
            edges_to_remove.append(edge)
        for edge in edges_to_remove:
            edges.remove(edge)
        visited.add(current)
    return len(visited)

top_three_orders = [0,0,0]

while len(edges):
    current_order = traverse_graph(edges)
    if current_order > min(top_three_orders):
        top_three_orders.remove(min(top_three_orders))
        top_three_orders.append(current_order)
print(reduce(lambda a,b: a*b, top_three_orders))

97384


In [66]:
import math
from itertools import combinations

points = [tuple(map(int, line.strip().split(','))) for line in lines]
# This is the part that takes up all of the computation time
edges = sorted(combinations(points, 2), key=lambda pair: math.dist(*pair))

class Graph:
    edges: set[tuple[tuple[int, int, int], tuple[int, int, int]]]
    nodes: set[tuple[int, int, int]]
    last_inserted_edge: tuple[tuple[int, int, int], tuple[int, int, int]]
    
    def __init__(self):
        self.edges = set()
        self.nodes = set()
        
    def add_edge(self, a: tuple[int, int, int], b: tuple[int, int, int]):
        self.edges.add((a, b))
        self.nodes.add(a)
        self.nodes.add(b)
        self.last_inserted_edge = (a, b)
        
    def merge(self, other: 'Graph'):
        self.edges.update(other.edges)
        self.nodes.update(other.nodes)
        
graphs: list[Graph] = []
for edge in edges:
    a, b = edge
    graph_with_a = next((g for g in graphs if a in g.nodes), None)
    graph_with_b = next((g for g in graphs if b in g.nodes), None)
    # Nodes of Edge are not in any graph
    if graph_with_a is None and graph_with_b is None:
        g = Graph()
        g.add_edge(a, b)
        graphs.append(g)
    # Node a is in a graph, node b is not
    elif graph_with_a is not None and graph_with_b is None:
        graph_with_a.add_edge(a, b)
    # Node b is in a graph, node a is not
    elif graph_with_a is None and graph_with_b is not None:
        graph_with_b.add_edge(a, b)
    # Both nodes are in different graphs
    elif graph_with_a is not graph_with_b:
        graph_with_a.merge(graph_with_b)
        graphs.remove(graph_with_b)
        graph_with_a.add_edge(a, b)
    # If both nodes are already in the same graph, do nothing
    
    # Check if we have a single graph containing all points
    if len(graphs) == 1 and len(graphs[0].nodes) == len(points):
        last_edge = graphs[0].last_inserted_edge
        print(last_edge)
        print(last_edge[0][0] * last_edge[1][0])
        break

((93722, 14018, 13354), (96068, 7627, 883))
9003685096


### NumPy can do it in half the time

In [67]:
import numpy as np

points_array = np.array(points)
num_points = points_array.shape[0]
r1, r2 = np.triu_indices(num_points, k=1)
pairs = np.column_stack((r1, r2))

diff = points_array[r1] - points_array[r2]
dists = np.linalg.norm(diff, axis=1)

sorted_indices = np.argsort(dists)
sorted_pairs_idx = pairs[sorted_indices]

graphs: list[Graph] = []
for edge_idx in sorted_pairs_idx:
    a, b = points_array[edge_idx]
    a, b = tuple(a), tuple(b)
    graph_with_a = next((g for g in graphs if a in g.nodes), None)
    graph_with_b = next((g for g in graphs if b in g.nodes), None)
    # Nodes of Edge are not in any graph
    if graph_with_a is None and graph_with_b is None:
        g = Graph()
        g.add_edge(a, b)
        graphs.append(g)
    # Node a is in a graph, node b is not
    elif graph_with_a is not None and graph_with_b is None:
        graph_with_a.add_edge(a, b)
    # Node b is in a graph, node a is not
    elif graph_with_a is None and graph_with_b is not None:
        graph_with_b.add_edge(a, b)
    # Both nodes are in different graphs
    elif graph_with_a is not graph_with_b:
        graph_with_a.merge(graph_with_b)
        graphs.remove(graph_with_b)
        graph_with_a.add_edge(a, b)
    # If both nodes are already in the same graph, do nothing
    
    # Check if we have a single graph containing all points
    if len(graphs) == 1 and len(graphs[0].nodes) == len(points):
        last_edge = graphs[0].last_inserted_edge
        print(last_edge)
        print(np.int64(last_edge[0][0]) * np.int64(last_edge[1][0]))
        break


((93722, 14018, 13354), (96068, 7627, 883))
9003685096
